
# Agent Telemetry & Dashboard Demo

This notebook illustrates how to **instrument a LangGraph agent** and collect:

- Node execution times  
- Simple event logs  
- A basic visualization of where time is spent  



## 1. Setup


In [ ]:

%pip install -q langgraph langchain-openai langchain matplotlib

import os, time
os.environ.setdefault("OPENAI_API_KEY", "sk-REPLACE_ME")



## 2. A Simple Two-Node Agent

We build:

- `analysis_node` — interprets the question  
- `answer_node` — generates the final response  

We wrap each node with a **telemetry decorator**.


In [ ]:

from langgraph.graph import StateGraph, START, END, MessagesState
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

telemetry_log = []

def record_event(node_name: str, start_time: float, end_time: float):
    telemetry_log.append({
        "node": node_name,
        "start": start_time,
        "end": end_time,
        "duration": end_time - start_time,
    })

def with_telemetry(node_name, fn):
    def wrapper(state: MessagesState):
        t0 = time.time()
        out = fn(state)
        t1 = time.time()
        record_event(node_name, t0, t1)
        return out
    return wrapper

def analysis_fn(state: MessagesState):
    q = state["messages"][0]["content"]
    prompt = f"Classify the user's request type (research, coding, planning, other) and extract key entities.\n\nRequest: {q}"
    resp = llm.invoke([{"role": "user", "content": prompt}])
    return {"messages": state["messages"] + [{"role": "assistant", "name": "analysis", "content": resp.content}]}

def answer_fn(state: MessagesState):
    analysis = state["messages"][-1]["content"]
    q = state["messages"][0]["content"]
    prompt = (
        "You are a helpful assistant. Using the analysis below, answer the original question clearly.\n\n"
        f"Analysis: {analysis}\n\n"
        f"Question: {q}"
    )
    resp = llm.invoke([{"role": "user", "content": prompt}])
    return {"messages": state["messages"] + [{"role": "assistant", "name": "answer", "content": resp.content}]}

analysis_node = with_telemetry("analysis", analysis_fn)
answer_node = with_telemetry("answer", answer_fn)



## 3. Build and Run the Graph


In [ ]:

builder = StateGraph(MessagesState)
builder.add_node("analysis", analysis_node)
builder.add_node("answer", answer_node)

builder.add_edge(START, "analysis")
builder.add_edge("analysis", "answer")
builder.add_edge("answer", END)

agent_graph = builder.compile()

query = "Design an agent that uses MCP tools and RAG to summarize internal documents daily."
state = {"messages": [{"role": "user", "content": query}]}

result = agent_graph.invoke(state)

print("Final answer:\n")
print(result["messages"][-1]["content"])



## 4. Visualize Telemetry

We now inspect the collected `telemetry_log` and plot node durations.


In [ ]:

import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(telemetry_log)
display(df)

plt.figure(figsize=(6,4))
plt.bar(df["node"], df["duration"])
plt.ylabel("Duration (s)")
plt.title("Node Execution Time")
plt.show()



## 5. Takeaways

- You can **wrap any node** with telemetry logging without changing its core logic.  
- Logs can be streamed to:
  - a DB (for production analytics)
  - a dashboard (Streamlit / Grafana)
  - alerting systems (Slack, PagerDuty)  

This pattern is the seed of a full **AgentOps dashboard**.
